# Direct Transcrition Methods
(Working in Progress) This tutorial follows optimization, and use direct transcription method to solve trajectory optimization problem.

The Nonlinear Programming Problem(NLP) is solve with help of casadi and IPOPT. 

In [1]:
import mujoco
import mediapy as media
from casadi import *

## Cartpole dynamics

In [2]:
xml = """"
<mujoco model="inverted pendulum">
  <compiler inertiafromgeom="true"/>
  <default>
    <joint armature="0" damping="1" limited="true"/>
    <geom contype="0" friction="1 0.1 0.1" rgba="0.4 0.33 0.26 1.0"/>
    <tendon/>
    <motor ctrlrange="-3 3"/>
  </default>
  <option gravity="0 0 -9.81" timestep="0.02" />
  <custom>
    <!-- brax custom params -->
    <numeric data="10000" name="constraint_stiffness"/>
    <numeric data="10000" name="constraint_limit_stiffness"/>
    <numeric data="0" name="spring_mass_scale"/>
    <numeric data="1" name="spring_inertia_scale"/>
    <numeric data="5" name="solver_maxls"/>
  </custom>
  <size nstack="3000"/>
  <worldbody>
    <light diffuse="1 1 1" pos="0 0 3" dir="0 0 -1"/>
    <!-- geom name="ground" type="plane" pos="0 0 0" / -->
    <geom name="rail" pos="0 0 0" quat="0.707 0 0.707 0" size="0.02 1" type="capsule" rgba="1 1 1 1"/>
    <body name="cart" pos="0 0 0">
      <joint axis="1 0 0" limited="false" name="slider" pos="0 0 0" type="slide"/>
      <geom name="cart" pos="0 0 0" quat="0.707 0 0.707 0" size="0.1 0.1" type="capsule"/>
      <body name="pole" pos="0 0 0">
        <joint axis="0 1 0" name="hinge" pos="0 0 0" limited="false" type="hinge"/>
        <geom fromto="0 0 0 0.001 0 0.6" name="cpole" size="0.049 0.3" type="capsule"/>
        <!--                 <body name="pole2" pos="0.001 0 0.6"><joint name="hinge2" type="hinge" pos="0 0 0" axis="0 1 0"/><geom name="cpole2" type="capsule" fromto="0 0 0 0 0 0.6" size="0.05 0.3" rgba="0.7 0 0.7 1"/><site name="tip2" pos="0 0 .6"/></body>-->
      </body>
    </body>
  </worldbody>
  <actuator>
    <motor ctrllimited="true" ctrlrange="-3 3" gear="100" joint="slider" name="slide"/>
  </actuator>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


## Casadi Wrap for mujoco dynamics

In [3]:
class CasadiMujocoWrap(Callback):
    def __init__(self, name, opts={}):
        Callback.__init__(self)
        self.construct(name, opts)

    # Number of inputs and outputs
    def get_n_in(self): return 2
    def get_n_out(self): return 1

    def get_sparsity_in(self,i):
        if i == 0:
            return Sparsity.dense(4,1)
        elif i == 1:
            return Sparsity.dense(1,1)
        else:
            raise ValueError("wrong args index")

    def get_sparsity_out(self,i):
        return Sparsity.dense(4,1)

    # Evaluate numerically
    def eval(self, arg):
        x = arg[0].full().flatten()
        u = arg[1].full().flatten()
        qpos, qvel = np.split(x, 2)
        data.qpos = qpos
        data.qvel = qvel
        data.ctrl = u
        mujoco.mj_step(model, data)
        x_next = np.hstack([data.qpos, data.qvel])
        return [DM(x_next)]

mujoco_dyanmics = CasadiMujocoWrap('f', {"enable_fd":True})

In [4]:
mujoco.mj_resetData(model, data) 

state = np.hstack([data.qpos, data.qvel])
u = np.array([1.0])
frames = []
while data.time < duration:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    if len(frames) < data.time * framerate:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)
media.show_video(frames, fps=framerate)

## Construct NLP and solve with IPOPT

In [5]:
T = 50

prob = Opti()
x = prob.variable(4, T+1)
u = prob.variable(1, T)

# initial condition
x0 = np.array([0, np.pi, 0, 0])
# target
x_target = np.array([0, 0, 0, 0])
# initial condition constraints
prob.subject_to(x[:, 0] == x0)
# terminal condition
prob.subject_to(x[:, -1] == x_target)
for i in range(T):
    # dynamics constraints
    prob.subject_to(x[:, i+1] == mujoco_dyanmics(x[:, i], u[:, i]))
# control limit
prob.subject_to([u>=-3, u<=3])
# just feasible problem, no objective
prob.minimize(0)

# set initial guess (it is crtical to obtain the solution in current NLP formulation)
x_init = np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)
prob.set_initial(x, x_init)

prob.solver("ipopt")
sol = prob.solve()
traj_x = np.swapaxes(sol.value(x), 0, 1)
traj_u = sol.value(u)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1208
Number of nonzeros in inequality constraint Jacobian.:      100
Number of nonzeros in Lagrangian Hessian.............:      750

Total number of variables............................:      254
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      208
Total number of inequality c

## Test results

In [6]:
frames = []
for state in traj_x:
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

In [7]:
mujoco.mj_resetData(model, data) 
state = x0
frames = []
for u in traj_u:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

## More NLPs

In [11]:
T = 50

prob = Opti()
x = prob.variable(4, T+1)
u = prob.variable(1, T)

# initial condition
x0 = np.array([0, np.pi, 0, 0])
# initial condition constraints
prob.subject_to(x[:, 0] == x0)

# target
x_target = np.array([0, 0, 0, 0])
# we do not set terminal condition but using running cost to push carpole up
Q = diag(SX([0.01, 1, 0.01, 0.01]))
R = diag(SX([0.01]))
x_sym = SX.sym("x", 4)
u_sym = SX.sym("u", 1)
cost_fn = Function('cost', [x_sym, u_sym], [(x_sym-x_target).T @ Q @ (x_sym-x_target) + u_sym.T @ R @ u_sym])
J = 0
for i in range(T):
    # dynamics constraints
    prob.subject_to(x[:, i+1] == mujoco_dyanmics(x[:, i], u[:, i]))
    J = J + cost_fn(x[:, i], u[:, i])
# control limit
prob.subject_to([u>=-3, u<=3])
# minimize running cost
prob.minimize(J)

# set initial guess (it is crtical to obtain the solution in current NLP formulation)
x_init = np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)
prob.set_initial(x, x_init)
prob.solver("ipopt", {}, {"max_iter": 100, "acceptable_iter": 100})

try:
    sol = prob.solve()
    traj_x = np.swapaxes(sol.value(x), 0, 1)
    traj_u = sol.value(u)
    print("Max it reached")
except:
    traj_x = np.swapaxes(prob.debug.value(x), 0, 1)
    traj_u = prob.debug.value(u)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      484
Number of nonzeros in inequality constraint Jacobian.:       40
Number of nonzeros in Lagrangian Hessian.............:      300

Total number of variables............................:      104
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       84
Total number of inequality constraints...............:       40
        inequality constraints with only lower bounds:       20
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       20

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  7.0814412e+01 4.47e-01 3.46e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [12]:
frames = []
for state in traj_x:
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

In [10]:
mujoco.mj_resetData(model, data) 
state = x0
frames = []
for u in traj_u:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)